In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms, datasets
import torchvision
from torch.utils.data import DataLoader,Dataset
from PIL import Image

import math
import time
import numpy as np

In [45]:
__all__ = ['darknet']
defaultcfg = {
    19 : [[32,3],'M',[64,3],'M',[128,3],[64,1],[128,3],'M',[256,3],[128,1],[256,3],'M',[512,3],[256,1],[512,3],[256,1],[512,3],'M',[1024,3],[512,1],[1024,3],[512,1],[1024,3]]
}

class darknet(nn.Module):
    def __init__(self, depth=19, init_weights=False, cfg=None):
        super(darknet, self).__init__()
        if cfg is None:
            cfg = defaultcfg[depth]

        self.feature = self.make_layers(cfg, True)
        num_classes = 30
        self.classifier = nn.Sequential(nn.Conv2d(cfg[-1][0], out_channels=num_classes, kernel_size=1, stride=1, padding=1, bias=False),\
                                        nn.AvgPool2d(7))
        if init_weights:
            self._initialize_weights()

    def make_layers(self, cfg, batch_norm=True):
        layers = []
        in_channels = 3
        for l in cfg:
            if l == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, out_channels=l[0], kernel_size=l[1], stride=1, padding=1, bias=False)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(l[0]), nn.LeakyReLU(0.1)]
                else:
                    layers += [conv2d, nn.LeakyReLU(0.1)]
                in_channels = l[0]
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.feature(x)
        print x.shape
        x = self.classifier(x)
        y = x.view(x.size(0), -1)
        return y

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(0.5)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

In [46]:
def get_layer_fm_index(index,conv_cfg):
    layer_index = 0
    fm_index = 0
    while (index >= conv_cfg[layer_index]):
        index = index - conv_cfg[layer_index]
        layer_index = layer_index + 1
    fm_index = index
    return layer_index,fm_index

def get_index(layer_index,fm_index,conv_cfg):
    index = 0
    for i in range (layer_index):
        index = index + conv_cfg[i]
    index = index + fm_index
    return index

def generate_pruned_cfg(loss_rank, cfg, prune_cnt):

    conv_cfg = []# store the fm cnt of each conv layer
    pruned_layer_cfg = [[]]# store the remain fm index of each conv layer
    pruned_model_cfg = cfg # store pruned cfg to generate new model

    for l in cfg:
        if l != 'M':
            conv_cfg.append(l[0])
    
    for i in range(len(cfg)-cfg.count('M')-1):
        pruned_layer_cfg.append([])
    
    fm_cnt=len(loss_rank) - prune_cnt#要保留的feature map个数

    for i in range(fm_cnt):
        layer_index,fm_index=get_layer_fm_index(loss_rank[i],conv_cfg)
        pruned_layer_cfg[layer_index].append(fm_index)
    
    layer_index = 0
    for i in range(len(cfg)):
        if cfg[i][0] != 'M':
            pruned_model_cfg[i][0]=len(pruned_layer_cfg[layer_index])
            layer_index+=1
    return pruned_model_cfg,pruned_layer_cfg

def get_layer_fm_index(index,conv_cfg):
    layer_index = 0
    fm_index = 0
    while (index >= conv_cfg[layer_index]):
        index = index - conv_cfg[layer_index]
        layer_index = layer_index + 1
    fm_index = index
    return layer_index,fm_index

def get_index(layer_index,fm_index,conv_cfg):
    index = 0
    for i in range (layer_index):
        index = index + conv_cfg[i]
    index = index + fm_index
    return index

def generate_pruned_cfg(loss_rank, cfg, prune_cnt):

    conv_cfg = []# store the fm cnt of each conv layer
    pruned_layer_cfg = [[]]# store the remain fm index of each conv layer
    pruned_model_cfg = cfg # store pruned cfg to generate new model

    for l in cfg:
        if l != 'M':
            conv_cfg.append(l[0])
    
    for i in range(len(cfg)-cfg.count('M')-1):
        pruned_layer_cfg.append([])
    
    fm_cnt=len(loss_rank) - prune_cnt#要保留的feature map个数

    for i in range(fm_cnt):
        layer_index,fm_index=get_layer_fm_index(loss_rank[i],conv_cfg)
        pruned_layer_cfg[layer_index].append(fm_index)
    
    layer_index = 0
    for i in range(len(cfg)):
        if cfg[i][0] != 'M':
            pruned_model_cfg[i][0]=len(pruned_layer_cfg[layer_index])
            layer_index+=1
    return pruned_model_cfg,pruned_layer_cfg

In [23]:
model = darknet()

In [24]:
loss = np.random.random(7200)
loss_rank = np.argsort(loss)
cfg=[[32,3],'M',[64,3],'M',[128,3],[64,1],[128,3],'M',[256,3],[128,1],[256,3],'M',[512,3],[256,1],[512,3],[256,1],[512,3],'M',[1024,3],[512,1],[1024,3],[512,1],[1024,3]]
pruned_model_cfg,pruned_layer_cfg = generate_pruned_cfg(loss_rank, cfg, 3600)

In [49]:
print pruned_model_cfg

[[15, 3], 'M', [31, 3], 'M', [68, 3], [41, 1], [67, 3], 'M', [148, 3], [67, 1], [138, 3], 'M', [251, 3], [120, 1], [243, 3], [130, 1], [233, 3], 'M', [523, 3], [258, 1], [520, 3], [243, 1], [504, 3]]


In [47]:
pruned_model = darknet(cfg=pruned_model_cfg)
print pruned_model

darknet(
  (feature): Sequential(
    (0): Conv2d(3, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(15, 31, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (5): BatchNorm2d(31, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.1)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(31, 68, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (9): BatchNorm2d(68, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.1)
    (11): Conv2d(68, 41, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
    (12): BatchNorm2d(41, eps=1e-05, momentum=0.1, affine=True, t

In [48]:
test(pruned_model,val_loader)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [40]:
def pruned_model_init(model, pruned_model, pruned_layer_cfg):
    conv_layer_index = 0
    for layer_index,pruned_layer in pruned_model.feature._modules.items():
        
        _,layer = model.feature._modules.items()[int(layer_index)]
        
        if isinstance(pruned_layer,nn.Conv2d):
            
            if(conv_layer_index == 0):
                weight=layer.weight.data.cpu().numpy()
                pruned_weight=layer.weight.data.cpu().numpy()[:len(pruned_layer_cfg[conv_layer_index])]
                print pruned_weight.shape
                
                out_channel=0
                for i in pruned_layer_cfg[conv_layer_index]:
                    pruned_weight[out_channel]=weight[i]
                    out_channel=out_channel+1
                
                pruned_layer.weight.data = torch.from_numpy(pruned_weight).cuda()
                
            else:
                weight=layer.weight.data.cpu().numpy()
                pruned_weight=layer.weight.data.cpu().numpy()[:len(pruned_layer_cfg[conv_layer_index]),:len(pruned_layer_cfg[conv_layer_index - 1])]
                print pruned_weight.shape
                
                in_channel=0
                out_channel=0
                for i in pruned_layer_cfg[conv_layer_index]:
                    for j in pruned_layer_cfg[conv_layer_index - 1]:
                        pruned_weight[out_channel,in_channel]=weight[i,j]
                        in_channel=in_channel+1
                    in_channel=0
                    out_channel=out_channel+1
                
                pruned_layer.weight.data = torch.from_numpy(pruned_weight).cuda()
            continue
            
        elif isinstance(pruned_layer,nn.BatchNorm2d):
            
            if(conv_layer_index < 18):
                
                pruned_weight=np.zeros(len(pruned_layer_cfg[conv_layer_index]))
                pruned_bias=np.zeros(len(pruned_layer_cfg[conv_layer_index]))
                pruned_mean=np.zeros(len(pruned_layer_cfg[conv_layer_index]))
                pruned_var=np.zeros(len(pruned_layer_cfg[conv_layer_index]))
                
                channel=0
                for i in pruned_layer_cfg[conv_layer_index]:
                    pruned_weight[channel]=layer.weight.data.cpu().numpy()[i]
                    pruned_bias[channel]=layer.bias.data.cpu().numpy()[i]
                    pruned_mean[channel]=layer.running_mean.data.cpu().numpy()[i]
                    pruned_var[channel]=layer.running_var.data.cpu().numpy()[i]
                    channel = channel + 1
                pruned_layer.weight.data = torch.from_numpy(pruned_weight).cuda()
                pruned_layer.bias.data = torch.from_numpy(pruned_bias).cuda()
                pruned_layer.running_mean.data = torch.from_numpy(pruned_mean).cuda()
                pruned_layer.running_var.data = torch.from_numpy(pruned_var).cuda()

            conv_layer_index += 1
        

In [41]:
model.load_state_dict(torch.load('darkenet19.pkl'))
pruned_model_init(model, pruned_model, pruned_layer_cfg)

(15, 3, 3, 3)
(31, 15, 3, 3)
(68, 31, 3, 3)
(41, 68, 1, 1)
(67, 41, 3, 3)
(148, 67, 3, 3)
(67, 148, 1, 1)
(138, 67, 3, 3)
(251, 138, 3, 3)
(120, 251, 1, 1)
(243, 120, 3, 3)
(130, 243, 1, 1)
(233, 130, 3, 3)
(523, 233, 3, 3)
(258, 523, 1, 1)
(520, 258, 3, 3)
(243, 520, 1, 1)
(504, 243, 3, 3)


In [3]:
batchsize = 32
train_data_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        #transforms.RandomRotation(10),
        #transforms.ColorJitter(hue=0.1,saturation=0.75),
        transforms.RandomHorizontalFlip(),
        #transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

tset_data_transform = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

train_dataset = torchvision.datasets.ImageFolder(root='/media/lulugay/PC/CCCV-30/train_set',transform=train_data_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchsize, shuffle=True, num_workers=12)
 
val_dataset = torchvision.datasets.ImageFolder(root='/media/lulugay/PC/CCCV-30/test_set', transform=tset_data_transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batchsize, shuffle=True, num_workers=12)

In [4]:
model = darknet()
model.cuda()
loss_func = nn.CrossEntropyLoss()

In [5]:
def test(net,data_loader):
    net.eval()
    corret,total = 0,0
    running_loss = 0.0
    for inputs,labels in data_loader:
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(Variable(inputs))
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        corret += (predicted == labels).sum()
        accuracy = 100 * float(corret) / float(total)
        loss = loss_func(outputs, labels)
        running_loss += loss.data[0]
    running_loss = running_loss / (15000 / batchsize)
    print('Test Accuracy: %.4f %%, Loss: %.4f' % (accuracy,running_loss))
    return accuracy,float(running_loss)

def poly(base_lr,power,total_epoch,now_epoch):
    return base_lr*(1-math.pow(float(now_epoch)/float(total_epoch),power))

def get_layer_fm_index(index):
    layer_index = 0
    fm_index = 0
    while (index >= model_config[layer_index]):
        index = index - model_config[layer_index]
        layer_index = layer_index + 1
    fm_index = index
    return layer_index,fm_index

def get_index(layer_index,fm_index):
    index = 0
    for i in range (layer_index):
        index = index + model_config[i]
    index = index + fm_index
    return index


In [ ]:
#model.load_state_dict(torch.load('epoch49_weight.pkl'))
num_epochs = 80
for epoch in range(0,num_epochs):
    model.train()
    batch_size_start = time.time()
    running_loss = 0.0
    corret,total = 0,0
    print('train epoch%d'%epoch)
    for i,(inputs, labels) in enumerate(train_loader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        inputs = Variable(inputs)
        lables = Variable(labels)
        lr=poly(0.01,4,num_epochs,epoch)
        optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9)
        optimizer.zero_grad()
        outputs = model(inputs)
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        corret += (predicted == labels).sum()
        loss = loss_func(outputs, labels)        #交叉熵
        loss.backward()
        optimizer.step()                          #更新权重
        running_loss += loss.data[0]
    accuracy = 100 * float(corret) / float(total)
    running_loss = running_loss / (45000 / batchsize)
    print('Train Accuracy: %.4f %%, Loss: %.4f' % (accuracy, running_loss))
    print('test epoch%d'%epoch)
    test(model,val_loader)
    
    
    print('epoch [%d/%d] need time %.4f' % (epoch + 1, num_epochs, time.time() - batch_size_start))

In [8]:
model_config = [32,64,128,64,128,256,128,256,512,256,512,256,512,1024,512,1024,512]
loss = np.random.rand(6172)
loss_rank = np.argsort(loss)
print loss_rank

[5745 2442 4241 ...  525 2430 5853]
